In [2]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)
from datetime import datetime, date, time, timezone
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

import mglearn
from IPython.display import display

import glob
import os

In [ ]:
# Jaccard Similarity
str1='jeffree-star-cosmetics-magic-star-setting-powder'
str2='jeffree-star-cosmetics-magic-star-luminous-setting-powder'

In [ ]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.split('-')) 
    b = set(str2.split('-'))
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
get_jaccard_sim(str1, str2)

In [40]:
df = pd.read_csv('/Users/elle.le/Workspace/BoxSize_local/new_data/all_item.csv')
df.drop(columns=['Unnamed: 0'], inplace = True)
print(df.shape)
df['name_split'] = df['product_name'].str.split(' ')
df.head(5)

(14855, 12)


,variant_id,dim_packaging_length_mm,dim_packaging_width_mm,dim_packaging_height_mm,item_vol,item_label,product_name,variant_name,name_string,brand_name,brand_id,weight,name_split
0,126,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Taupe,anastasia-beverly-hills-perfect-brow-pencil-taupe,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"
1,127,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Blonde,anastasia-beverly-hills-perfect-brow-pencil-bl...,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"
2,128,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Caramel,anastasia-beverly-hills-perfect-brow-pencil-ca...,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"
3,129,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Medium Brown,anastasia-beverly-hills-perfect-brow-pencil-me...,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"
4,130,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Dark Brown,anastasia-beverly-hills-perfect-brow-pencil-da...,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"


In [64]:
#df_unique_name = df.drop_duplicates(subset='product_name')
df_unique_name = df.dropna(subset=['name_split','weight'])
df_unique_name.reset_index(inplace=True,drop=True)

In [65]:
df_unique_name

,variant_id,dim_packaging_length_mm,dim_packaging_width_mm,dim_packaging_height_mm,item_vol,item_label,product_name,variant_name,name_string,brand_name,brand_id,weight,name_split
0,126,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Taupe,anastasia-beverly-hills-perfect-brow-pencil-taupe,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"
1,127,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Blonde,anastasia-beverly-hills-perfect-brow-pencil-bl...,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"
2,128,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Caramel,anastasia-beverly-hills-perfect-brow-pencil-ca...,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"
3,129,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Medium Brown,anastasia-beverly-hills-perfect-brow-pencil-me...,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"
4,130,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Dark Brown,anastasia-beverly-hills-perfect-brow-pencil-da...,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14743,68804,NaN,NaN,NaN,NaN,NaN,Magic Star Luminous Setting Powder,Topaz,jeffree-star-cosmetics-magic-star-luminous-set...,Jeffree Star Cosmetics,2944.0,0.17,"[Magic, Star, Luminous, Setting, Powder]"
14744,68805,NaN,NaN,NaN,NaN,NaN,Magic Star Luminous Setting Powder,Honey,jeffree-star-cosmetics-magic-star-luminous-set...,Jeffree Star Cosmetics,2944.0,0.17,"[Magic, Star, Luminous, Setting, Powder]"
14745,68806,NaN,NaN,NaN,NaN,NaN,Magic Star Luminous Setting Powder,Suede,jeffree-star-cosmetics-magic-star-luminous-set...,Jeffree Star Cosmetics,2944.0,0.17,"[Magic, Star, Luminous, Setting, Powder]"
14746,68807,NaN,NaN,NaN,NaN,NaN,Magic Star Luminous Setting Powder,Caramel,jeffree-star-cosmetics-magic-star-luminous-set...,Jeffree Star Cosmetics,2944.0,0.17,"[Magic, Star, Luminous, Setting, Powder]"


In [66]:
total_word_list =[]
for i in range(df_unique_name.shape[0]):
    total_word_list += df_unique_name.iloc[i,-1] #Make sure the last columns is the name_split column

In [67]:
unique_word = np.unique(total_word_list)

In [68]:
len(unique_word)

3693

In [69]:
df.shape[0]

14855

In [70]:
name_matrix = np.zeros((df_unique_name.shape[0], len(unique_word)))

In [71]:
name_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [72]:
word_col_map = dict(zip(unique_word,range(len(unique_word))))

In [73]:
word_col_map['#11']

8

In [74]:
for i in range(df_unique_name.shape[0]):
    words = df_unique_name.iloc[i,-1]
    for word in words:
        name_matrix[i,word_col_map[word]] = 1
        
name_matrix 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [75]:
type(name_matrix)

numpy.ndarray

In [76]:
feature_df=pd.DataFrame(name_matrix)
feature_df

,0,1,2,3,4,5,6,7,8,9,...,3683,3684,3685,3686,3687,3688,3689,3690,3691,3692
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
feature_df[['variant_id','weight']] = df_unique_name[['variant_id','weight']]
feature_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,3685,3686,3687,3688,3689,3690,3691,3692,variant_id,weight
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126,0.03
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127,0.03
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,128,0.03


In [78]:
feature_df['weight'] = feature_df['weight']*1000

In [79]:
feature_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,3685,3686,3687,3688,3689,3690,3691,3692,variant_id,weight
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126,30.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127,30.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,128,30.0


In [80]:
feature_df.isna().sum()

0             0
1             0
2             0
3             0
4             0
             ..
3690          0
3691          0
3692          0
variant_id    0
weight        0
Length: 3695, dtype: int64

In [84]:
#KNN
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=10, algorithm='brute').fit(feature_df)
distances, indices = nbrs.kneighbors(feature_df)
indices

array([[    0,     1,     2, ...,    16,    12,    13],
       [    1,     2,     0, ...,    16,    12,    13],
       [    2,     1,     3, ...,    16,    12,    13],
       ...,
       [14745, 14744, 14746, ..., 14739, 14738, 14718],
       [14746, 14745, 14744, ..., 14739, 14738, 14718],
       [14747, 14724, 14723, ..., 14746, 14745, 14744]])

In [83]:
distances

array([[  0.        ,   1.        ],
       [  0.        ,   1.        ],
       [  0.        ,   1.        ],
       ...,
       [  0.        ,   1.        ],
       [  0.        ,   1.        ],
       [  0.        , 305.66484914]])

In [90]:
df_unique_name.iloc[14738,6:12]

product_name                   Magic Star Luminous Setting Powder
variant_name                                          Translucent
name_string     jeffree-star-cosmetics-magic-star-luminous-set...
brand_name                                 Jeffree Star Cosmetics
brand_id                                                     2944
weight                                                       0.17
Name: 14738, dtype: object

In [91]:
df_unique_name.iloc[14718,6:12]

product_name          UNIT 502 Loose Setting Powder
variant_name          UNIT 502 Loose Setting Powder
name_string     units-unit-502-loose-setting-powder
brand_name                                    UNITS
brand_id                                       3021
weight                                         0.18
Name: 14718, dtype: object

In [93]:
df_unique_name.iloc[14746,6:12]

product_name                   Magic Star Luminous Setting Powder
variant_name                                              Caramel
name_string     jeffree-star-cosmetics-magic-star-luminous-set...
brand_name                                 Jeffree Star Cosmetics
brand_id                                                     2944
weight                                                       0.17
Name: 14746, dtype: object

In [94]:
df_unique_name.iloc[14745,6:12]

product_name                   Magic Star Luminous Setting Powder
variant_name                                                Suede
name_string     jeffree-star-cosmetics-magic-star-luminous-set...
brand_name                                 Jeffree Star Cosmetics
brand_id                                                     2944
weight                                                       0.17
Name: 14745, dtype: object

In [25]:
name_matrix[0,685]

1.0

In [22]:
word_col_map['Perfect']

2477

In [23]:
word_col_map['Brow']

685